In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
import time
from functools import reduce
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)
from pprint import pprint
import itertools

In [8]:
product_data = pd.read_csv('data/AnnualOptimizerInputData.csv')
last_quarter_product_base = pd.read_csv('data/LastQuarterData.csv')
coefficient_matrix = pd.read_csv('data/coefficient_matrix.csv')

products = product_data.shape[0]
idx = range(products)
weeks = 52
tot = products*weeks

init_data = pd.read_csv('data/Weekly_optimiser_2_results.csv')
init_data['edlp_discount'] = init_data['EDLP_Price']/init_data['BasePrice']
init_data['BasePrice'] = init_data['BasePrice'].astype(float)
# base_price_def = init_data['BasePrice'].values.reshape((6,52))
base_price = []
for i in range(products):
    base_price.append(init_data.iloc[i::6, 9].values.tolist())
    
edlp = []
for i in range(products):
    edlp.append(init_data.iloc[i::6, 12].values.tolist())

tpr = []
for i in range(products):
    tpr.append(init_data.iloc[i::6, 8].values.tolist())

flag = []
for i in range(products):
    flag.append(init_data.iloc[i::6, 5].values.tolist())



coef = []
cols = coefficient_matrix.columns
coefficient_matrix_dr = coefficient_matrix.drop(columns=['Modelled_Product','wk'])#'Product1_Retailer_EDLP_Product6_Retailer_EDLP'])

for i in range(products):
    coef.append(coefficient_matrix_dr.iloc[i::6, :].values.tolist())

print('Shape=', len(coef), len(coef[0]), len(coef[0][0]))

base = last_quarter_product_base['Base Price']
# base_price = [[x]*weeks for x in base]

target_trade_spend = product_data.Product_Total_Spend.tolist()
total_target_trade_spend_final = sum(target_trade_spend)
target_edlp_spend = product_data.Product_EDLP_Spend.tolist()
target_tpr_spend = product_data.Product_TPR_Spend.tolist()
target_tpr_events = product_data.TPR_Events.tolist()

target_trade_spend_lower_bd = [x * 0.9 for x in target_trade_spend]
target_trade_spend_upper_bd = [x * 1.1 for x in target_trade_spend]

target_edlp_spend_lower_bd = [x * 0.8 for x in target_edlp_spend]
target_edlp_spend_upper_bd = [x * 1.2 for x in target_edlp_spend]

target_tpr_spend_lower_bd = [x * 0.8 for x in target_tpr_spend]
target_tpr_spend_upper_bd = [x * 1.2 for x in target_tpr_spend]

target_tpr_events_lb = product_data.TPR_Events_Min.tolist()
target_tpr_events_ub = product_data.TPR_Events_Max.tolist()

Shape= 6 52 14


In [5]:

products=2
weeks=3
tot=products*weeks
coef_p1 = [[-6,0.01,1,-0.003,0.67,23],
           [-6,0.01,1,-0.003,0.67,23],
           [-6,0.01,1,-0.003,0.67,23]]

coef_p2 = [[1.2,-0.004,-5,0.02,-5.73,20],
           [1.2,-0.004,-5,0.02,-5.73,21],
           [1.2,-0.004,-5,0.05,-5.73,20]]

coef = [coef_p1,coef_p2]

base_price_1=[10,10,10]
base_price_2=[10,10,10]
base_price = [base_price_1, base_price_2]

edlp_p1=[0.95,1,1]
edlp_p2 =[1,1,0.96]
edlp = [edlp_p1,edlp_p2]

tpr_p1=[0,5,15]
tpr_p2 =[15,5,0]
tpr_p1=[0,0.05,0.15]
tpr_p2 =[0.15,0.05,0]

tpr = [tpr_p1,tpr_p2]

flag_p1=[1,0,0]
flag_p2=[0,0,1]
flag = [flag_p1,flag_p2]
print('Shape=', len(coef), len(coef[0]), len(coef[0][0]))


Shape= 2 3 6


In [12]:
def cal_sales_prior(edlp,tpr,flag,coef,j):
    unit_sales = []
    for i in  range(weeks):
        edlp_sum = 0
        tpr_sum = 0
        total = 0
        cc= 0
        for p in range(products):
            if p==j:
                z = 0
                edlp_sum+= (np.log( base_price[p][i] * edlp[p][i])*coef[i][cc])* z
                tpr_sum+= (np.log(base_price[p][i])*coef[i][cc]+ 100*tpr[p][i]*z)*(1- z)
            else:
                edlp_sum+= (np.log( base_price[p][i] * edlp[p][i])*coef[i][cc])* flag[p][i]
                tpr_sum+= (np.log(base_price[p][i])*coef[i][cc]+ 100*tpr[p][i]*coef[i][cc+1])*(1- flag[p][i])
                
            cc+=2
            
        unit_sales.append(np.exp(edlp_sum+ tpr_sum +coef[i][-1]))                 
    return unit_sales

def calculate_interaction(prd_no, inter_prd, edlp, flag,i):

    p1 = base_price[prd_no][i] * (1-flag[prd_no][i]) + base_price[prd_no][i]* edlp[prd_no][i] * (flag[prd_no][i])
    p2 = base_price[inter_prd][i] * (1-flag[inter_prd][i]) + base_price[inter_prd][i]* edlp[inter_prd][i] * (flag[inter_prd][i])
    
#     print(p1, p2,'*')
    return p1*p2
    
def cal_sales(edlp,tpr,flag,coef,j,factors):
    unit_sales = []
    for i in  range(weeks):
        print(edlp[i],coef[0],flag[i])
        
        edlp_sum = 0
        tpr_sum = 0
        cc= 0
        for p in range(products):
#             print(coef[i][cc])
            edlp_sum+= (np.log( base_price[p][i] * edlp[p][i])*coef[i][cc+1])* flag[p][i]
            tpr_sum+= (np.log(base_price[p][i])*coef[i][cc+1]+ 100*tpr[p][i]*coef[i][cc])*(1- flag[p][i])
            cc+=2
        try:
            interaction = calculate_interaction(j, factors[j], edlp,flag,i)
        except:
            interaction = 1
        unit_sales.append(np.exp(edlp_sum+ tpr_sum+ np.log(interaction)*coef[i][-2] + coef[i][-1]))                 
   
    return unit_sales



In [13]:
# initial = calc_dollar_sales(edlp,tpr,flag,coef,True)
# final = calc_dollar_sales(edlp,tpr,flag,coef)

# print('Shape=', len(coef), len(coef[0]), len(coef[0][0]))
print()
interaction_factors = {}
interaction_factors[0] = 5
interaction_factors[5] = 0

k = 0
print(cal_sales(edlp,tpr,flag,coef[k],k,interaction_factors), calc_price(edlp,tpr,flag,k))
k = 1
# print(cal_sales(edlp,tpr,flag,coef[k],k,interaction_factors), calc_price(edlp,tpr,flag,k))
# print(initial)
# print(final)
# result = []

# for i in range(weeks):
#     for j in range(products):
#         result.append(final[j][i])
        
# # print(result)
# pd.DataFrame(data=result)

# print()
# print(sum(calculate_lift(final, initial)))
# print()
# print(calculate_total_tpr_events(flag))
# z = calc_total_trade_spent(edlp,tpr,flag,coef)
# x = calc_edlp_trade_spent(edlp,tpr,flag,coef)
# print(z)
# print(x)
# calculate_tpr_trade_spend(z,x)


[0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593, 0.9949748743718593

IndexError: list index out of range

In [4]:
def calc_price(edlp,tpr,flag,j):
    price = []
    for i in range(weeks):
        price.append((base_price[j][i]*(1-tpr[j][i])*(1-flag[j][i]))  +  (base_price[j][i]*edlp[j][i]*flag[j][i]))
                   
    return price
                   
def calc_dollar_sales(edlp,tpr,flag,coef, prior=False):
    dollar_sales=[]
    for j in range(products):
        if not prior:
            unit_sales= cal_sales(edlp,tpr,flag,coef[j],j)
            price = calc_price(edlp,tpr,flag,j)
        else:
            unit_sales = cal_sales_prior(edlp,tpr,flag,coef[j],j)
            price = base_price[j]

        dollar_sales_single = [unit_sales[i]*price[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
    
    return dollar_sales

def calc_total_trade_spent(edlp,tpr,flag,coef):
    total_trade_spent = []
    for j in range(products):
        unit_sales= cal_sales(edlp,tpr,flag,coef[j],j)
        price = calc_price(edlp,tpr,flag,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp,tpr,flag,coef):
    edlp_trade_spent = []
#     flag = [flag_p1,flag_p2]
    for j in range(products):
        unit_sales= cal_sales(edlp,tpr,flag,coef[j],j)
        price = calc_price(edlp,tpr,flag,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]

        edlp_trade_spent_partial = [total_trade_spent_partial[i]*flag[j][i] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent


def calculate_lift(dollar_sales, prior_prom_sales):
    lift = []
    for i in range(products):
        for j in range(weeks):
            lift.append(dollar_sales[i][j] - prior_prom_sales[i][j])
    return lift

def calculate_total_tpr_events(flag):
    tpr_events = []
    for i in range(products):
        temp_sum = 0
        for j in range(weeks):
            temp_sum+=flag[i][j]
        tpr_events.append(weeks-temp_sum)
    return tpr_events

def calculate_tpr_trade_spend(total_trade_spent, edlp_trade_spent):
    tpr_spend =[]
    for i in range(products):
        tpr_spend.append([a-b for a,b in zip(total_trade_spent[i], edlp_trade_spent[i])])
    return tpr_spend